In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Image size and batch size
picture_size = 48
batch_size = 32

# Train data generator with augmentation
datagen_train = ImageDataGenerator(
    rescale=1./255, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True
)

train_set = datagen_train.flow_from_directory(
    'C:/Users/iStudio/Downloads/DATAS/images/train',  # Path to training data
    target_size=(picture_size, picture_size),
    batch_size=batch_size,
    color_mode='grayscale',  # Grayscale images
    class_mode='categorical'  # Multi-class classification
)

# Validation data generator
datagen_val = ImageDataGenerator(rescale=1./255)

validation_set = datagen_val.flow_from_directory(
    'C:/Users/iStudio/Downloads/DATAS/images/validation',  # Path to validation data
    target_size=(picture_size, picture_size),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical'
)

# Test data generator for evaluation (optional)
datagen_test = ImageDataGenerator(rescale=1./255)
test_set = datagen_test.flow_from_directory(
    'C:/Users/iStudio/Downloads/DATAS/images/test',  # Path to test data
    target_size=(picture_size, picture_size),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False
)

# Verify class labels
print("Train classes:", train_set.class_indices)
print("Validation classes:", validation_set.class_indices)
print("Test classes:", test_set.class_indices)

Found 8931 images belonging to 2 classes.
Found 2099 images belonging to 2 classes.
Found 2099 images belonging to 2 classes.
Train classes: {'angry': 0, 'sad': 1}
Validation classes: {'angry': 0, 'sad': 1}
Test classes: {'angry': 0, 'sad': 1}


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define a sequential model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(picture_size, picture_size, 1)),  # Grayscale input
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),

    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout to prevent overfitting

    Dense(2, activation='softmax')  # Output layer, assuming 2 classes (adjust based on your dataset)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model architecture
model.summary()


c:\Users\iStudio\.conda\envs\bud\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       819,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 838,402 (3.20 MB)

 Trainable params: 838,402 (3.20 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Early stopping callback to stop training when validation loss does not improve
early_stopping = EarlyStopping(monitor='val_loss', patience=10000000, restore_best_weights=True)

# Model checkpoint callback to save the best model
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(
    train_set,
    epochs=500,  # Train for 100 epochs
    validation_data=validation_set,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/500
280/280 ━━━━━━━━━━━━━━━━━━━━ 41s 143ms/step - accuracy: 0.7540 - loss: 0.5037 - val_accuracy: 0.7156 - val_loss: 0.5656
Epoch 2/500
280/280 ━━━━━━━━━━━━━━━━━━━━ 40s 140ms/step - accuracy: 0.7584 - loss: 0.4927 - val_accuracy: 0.7261 - val_loss: 0.5604
Epoch 3/500
280/280 ━━━━━━━━━━━━━━━━━━━━ 28s 100ms/step - accuracy: 0.7554 - loss: 0.4887 - val_accuracy: 0.7194 - val_loss: 0.5614
Epoch 4/500
280/280 ━━━━━━━━━━━━━━━━━━━━ 39s 138ms/step - accuracy: 0.7571 - loss: 0.4906 - val_accuracy: 0.7213 - val_loss: 0.5716
Epoch 5/500
280/280 ━━━━━━━━━━━━━━━━━━━━ 32s 111ms/step - accuracy: 0.7625 - loss: 0.4763 - val_accuracy: 0.7189 - val_loss: 0.5687
Epoch 6/500
280/280 ━━━━━━━━━━━━━━━━━━━━ 49s 172ms/step - accuracy: 0.7645 - loss: 0.4759 - val_accuracy: 0.7118 - val_loss: 0.5576
Epoch 7/500
280/280 ━━━━━━━━━━━━━━━━━━━━ 39s 135ms/step - accuracy: 0.7727 - loss: 0.4695 - val_accuracy: 0.7194 - val_loss: 0.5617
Epoch 8/500
280/280 ━━━━━━━━━━━━━━━━━━━━ 39s 138ms/step - accuracy: 0.7694 -

In [6]:
# Save the model
model.save('new_114.h5')